### Notice!!! Please use search function to check all the cells with "notice" keywords, those are sanity checks and potential manual variable changes

In [0]:
#########################################################################################################
# Please define the target Timestamp for the end date for the generation of COVID patients dataset
# Notice: Format: YYYY-MM-DD
# Current set to 2021-12-10
############################################################################################################
# check to make sure they are consistent
# Previous dates used:
# 1. "2021-12-31", "20211231"
# 2. "2022-02-18", "20220218"
# 3. end_date, file_date = "2021-12-25", "20211225"
# end_date, file_date = "2021-12-21", "20211221"

start_date, end_date, file_date = "2020-03-01", "2021-12-25", "20211225_Lancet_bmi"
# register it so can be directly used in the following SQL statement
spark.conf.set("enddate.var", end_date)

In [0]:
#pip install xgboost
#pip install joblib
#pip install scikit-learn
#pip install shap

### Keep the follow cell empty can copy all the install commands and run them one by one

In [0]:
import numpy as np
import pandas as pd
import shap

## Scikit-learn related
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.linear_model import SGDClassifier
# from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
## from sklearn import cross_validation, metrics   #Additional scklearn functions, deprecated since version 0.18
## from sklearn.grid_search import GridSearchCV   #!!!the grid search package that has issue, dont use it
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from scipy import interp
from scipy import stats

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

########################################
## Import those from pyspark.ml
import pyspark.ml
import pyspark.sql.functions as F

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-

In [0]:
imids_training_df = spark.sql("""SELECT * FROM rdp_phi_sandbox.qw_IMID_COVID_trainset_cond_med_vax_{}""".format(file_date)) 
# imids_training_df.limit(5).toPandas()

In [0]:
# imids_training_df.limit(20).toPandas()

In [0]:
imids_training_df = imids_training_df.dropDuplicates()
# imids_training_df.limit(5).toPandas()

In [0]:
## Drop not used columns
col_to_drop = ('age_range', 'ethnicity', 'race1', 'race_v2', 'ethnicity_race', 'CVX_name', 'decided_index_date', 'CVX_name')
imids_training_df = imids_training_df.drop(*col_to_drop)

## rename IMIDs drugs columns
imids_training_df = imids_training_df.withColumnRenamed('prior_91_days_hydroxychloroquine_logic', 'hydroxychloroquine').withColumnRenamed('prior_91_days_methotrexate_logic', 'methotrexate')\
.withColumnRenamed('prior_91_days_leflunomide_teriflunomide_logic', 'leflunomide_teriflunomide').withColumnRenamed('prior_91_days_5_ASAa_logic', '5_ASA')\
.withColumnRenamed('prior_91_days_azathioprine_logic', 'azathioprine').withColumnRenamed('prior_91_days_mercaptopurine_logic', 'mercaptopurine')\
.withColumnRenamed('prior_91_days_mitoxantrone_logic', 'mitoxantrone')\
.withColumnRenamed('prior_91_days_mycophenolate_logic', 'mycophenolate').withColumnRenamed('prior_91_days_calcineurin_inhibitor_logic', 'calcineurin_inhibitor')\
.withColumnRenamed('prior_91_days_TNF_alpha_inhibitor_logic', 'TNF_alpha_inhibitor').withColumnRenamed('prior_91_days_fumarates_logic', 'fumarates')\
.withColumnRenamed('prior_91_days_interferons_logic', 'interferons').withColumnRenamed('prior_91_days_alkylating_agent_logic', 'alkylating_agent')\
.withColumnRenamed('prior_91_days_hydroxyurea_logic', 'hydroxyurea').withColumnRenamed('prior_91_days_dapsone_logic', 'dapsone')\
.withColumnRenamed('prior_91_days_cladribine_logic', 'cladribine').withColumnRenamed('prior_91_days_IL1_inhibitor_logic', 'IL1_inhibitor')\
.withColumnRenamed('prior_91_days_IL6_inhibitor_logic', 'IL6_inhibitor').withColumnRenamed('prior_91_days_IL12_23_inhibitor_logic', 'IL12_23_inhibitor')\
.withColumnRenamed('prior_91_days_IL17_inhibitor_logic', 'IL17_inhibitor')\
.withColumnRenamed('prior_91_days_IL23_inhibitor_logic', 'IL23_inhibitor')\
.withColumnRenamed('prior_91_days_abatacept_logic', 'abatacept').withColumnRenamed('prior_91_days_anti_BLyS_logic', 'anti_BLyS')\
.withColumnRenamed('prior_91_days_S1P_receptor_modulator_logic', 'S1P_receptor_modulator').withColumnRenamed('prior_91_days_JAK_inhibitor_logic', 'JAK_inhibitor')\
.withColumnRenamed('prior_91_days_integrin_inhibitor_logic', 'integrin_inhibitor').withColumnRenamed('prior_91_days_PDE4i_targeted_synthetic_logic', 'PDE4i_targeted_synthetic')\
.withColumnRenamed('prior_91_days_anti_CD20_logic', 'anti_CD20').withColumnRenamed('prior_91_days_anti_CD52_logic', 'anti_CD52')\
.withColumnRenamed('prior_91_days_budesonide_logic', 'budesonide').withColumnRenamed('prior_91_days_systemic_glucocorticoids_logic', 'systemic_glucocorticoids')\
.withColumnRenamed('after_10_days_monoclonal_antibody_covid_19_logic', 'monoclonal_antibody_covid_19')

In [0]:
## Temp solution, filter the positive patient here
pos_df = imids_training_df.filter(imids_training_df['results'] == "Positive")

## convert to pandas dataframe
# imids_training_pd_df = imids_training_onlyPos_df.toPandas()
 
## col names before dropping any
# list(imids_training_pd_df.columns)

In [0]:
## One hot-encode sex
categ = pos_df.select('sex').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('sex') == cat,1).otherwise(0)\
            .alias(str("sex_" + cat)) for cat in categ]
pos_one_hot_df = pos_df.select(pos_df.columns + exprs)

## Fill none values in the vaccination column
pos_one_hot_df = pos_one_hot_df.fillna({'Vaccination_status':'Not'})

## One hot-encode vaccination
categ = pos_one_hot_df.select('Vaccination_status').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('Vaccination_status') == cat,1).otherwise(0)\
            .alias(str("vaccination_" + cat)) for cat in categ]
pos_one_hot_df = pos_one_hot_df.select(pos_one_hot_df.columns + exprs)

## Drop not needed cols and original cols
pos_one_hot_df = pos_one_hot_df.drop("sex").drop("sex_Female").drop("sex_Unknown").drop("Vaccination_status").drop("vaccination_Not")

## change the name of those vaccination related columns
pos_one_hot_df = pos_one_hot_df.withColumnRenamed('vaccination_Fully', 'fully_vaccinated').withColumnRenamed('vaccination_Booster', 'boosted')

In [0]:
# ## Manually define the min max process
# from pyspark.sql.functions import max, min, mean, col

# max_age, min_age = pos_one_hot_df.select(max("age"), min("age")).first()
# pos_one_hot_minmax_df = pos_one_hot_df.withColumn("age_normalized", (col("age") - min_age) / (max_age - min_age) )
# pos_one_hot_minmax_df = pos_one_hot_minmax_df.drop("age")

In [0]:
# ## get the data where monoclonal_antibody_covid_19 = 0
# pos_one_hot_noAntibody_df = pos_one_hot_df.where(pos_one_hot_df.monoclonal_antibody_covid_19 == 0)

In [0]:
train_df = pos_one_hot_df.select("*")
train_df = train_df.drop("pat_id").drop("results")

## Drop SVI and geocoding features due to high missing percentage
## list: 'SVI_Socioeconomic', 'SVI_Household_Composition_Disability', 'SVI_Minority_Status_Language', 'SVI_Housing_Type_Transportation', 'SVI', 'Metro_area', 'Low_education', 'Low_employment',
## not including 'obesity'

train_df = train_df.select('patient_id', 'hospitalized_after_positive','IMV_after_positive','death_after_positive',
  'age', 'BMI', 'sex_Male',
  'hypertension', 'diabetes_type1and2', 'atrial_fibrillation', 'coronary_artery_disease', 'heart_failure', 'chronic_kidney_disease', 'copd', 'chronic_liver_disease', 'malignant_neoplastic_disease', 'asthma', 'HIV', 'history_transplant', 'stroke', 'opioid_dependence', 'fully_vaccinated', 'boosted', 
 'ibd', 'rheumatoid_arthritis', 'multiple_sclerosis','psoriatic_arthritis', 'psoriasis', 'systemic_sclerosis', 'spondyloarthritis', 'systemic_lupus', 'vasculitis', 'sarcoidosis', 'APS', 'sjogren_syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide_teriflunomide',
 '5_ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin_inhibitor',
 'TNF_alpha_inhibitor',
 'fumarates',
 'interferons',
 'alkylating_agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL1_inhibitor',
 'IL6_inhibitor',
 'IL12_23_inhibitor',
 'IL17_inhibitor',
 'IL23_inhibitor',
 'abatacept',
 'anti_BLyS',
 'S1P_receptor_modulator',
 'JAK_inhibitor',
 'integrin_inhibitor',
 'PDE4i_targeted_synthetic',
 'anti_CD20',
 'anti_CD52',
 'budesonide',
 'systemic_glucocorticoids',
 'monoclonal_antibody_covid_19')

In [0]:
train_pd_df = train_df.toPandas()
## change the patient id col to be the index of the training data
train_pd_df = train_pd_df.set_index("patient_id")
# display(train_pd_df)

## Training data preprocessing and test data split
* Assign features for training data to X
* Assign response to Y
* Define the number of CV & random seed used for tuning

In [0]:
# ### Uncomment to save table as backup for publications
# spark.sql("""DROP TABLE IF EXISTS rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}""".format(file_date))
# table_name = "rdp_phi_sandbox.qw_IMIDs_COVID_paper_train_data_processed_{}".format(file_date)
# ## Convert into Spark DataFrame
# spark_df_one_hot_encoded = spark.createDataFrame(df_one_hot_encoded)

# ## Write the table
# # spark_df_one_hot_encoded.write.mode("overwrite").saveAsTable(table_name)

In [0]:
#######################################################
#Random search CV method
#and
#Multi class roc_auc score method
########################################################
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer, roc_auc_score

###############################################################################################
#Binary class roc auc score method
#input: y_true, true labels from test fold
#       y_score, predicted probability on test fold
#       average, string, [None, ‘micro’, ‘macro’ (default), ‘samples’, ‘weighted’]
#                'macro': Calculate metrics for each label, and find their unweighted mean. 
#                This does not take label imbalance into account.
#                'weighted': Calculate metrics for each label, and find their average, 
#                weighted by support (the number of true instances for each label).
#output: auroc value for each class
#############################################################################################
def binary_class_roc_auc_score(y_true, y_score, average="weighted"):

    return roc_auc_score(y_true, y_score, average=average)

binaryclass_score = make_scorer(binary_class_roc_auc_score, needs_threshold = True)

In [0]:
## not including: , 'obesity'
train_pd_df.columns = ['hospitalized_after_positive', 'IMV_after_positive', 'death_after_positive', 'age', 'BMI', 'sex:male', 'hypertension', 'diabetes (type 1+2)', 'atrial fibrillation', 'coronary artery disease', 'heart failure', 'chronic kidney disease', 'COPD', 'chronic liver disease', 'malignant neoplastic disease', 'asthma', 'HIV', 'history of transplant', 'stroke', 'opioid dependence', 'fully vaccinated', 'boosted', 
 'inflammatory bowel disease', 'rheumatoid arthritis', 'multiple sclerosis','psoriatic arthritis', 'psoriasis', 'systemic sclerosis', 'spondyloarthritis', 'systemic lupus', 'vasculitis', 'sarcoidosis', 'antiphospholipid syndrome', 'Sjögren syndrome',
 'hydroxychloroquine',
 'methotrexate',
 'leflunomide teriflunomide',
 '5-ASA',
 'azathioprine',
 'mercaptopurine',
 'mitoxantrone',
 'mycophenolate',
 'calcineurin inhibitor',
 'TNF-α inhibitor',
 'fumarates',
 'interferons',
 'alkylating agent',
 'hydroxyurea',
 'dapsone',
 'cladribine',
 'IL-1 inhibitor',
 'IL-6 inhibitor',
 'IL-12/23 inhibitor',
 'IL-17 inhibitor',
 'IL-23 inhibitor',
 'abatacept',
 'anti-BLyS',
 'S1P receptor modulator',
 'JAK inhibitor',
 'integrin inhibitor',
 'PDE4i targeted synthetic',
 'anti-CD20',
 'anti-CD52',
 'budesonide',
 'systemic glucocorticoids', 'monoclonal antibody covid-19']

In [0]:
## Columns to drop based on VIF results or missingness
# train_pd_df.drop(['HIV'], axis = 1, inplace = True)
# train_pd_df.drop(['systemic sclerosis'], axis = 1, inplace = True)
# train_pd_df.drop(['leflunomide teriflunomide'], axis = 1, inplace = True)
# train_pd_df.drop(['5-ASA'], axis = 1, inplace = True)
train_pd_df.drop(['mitoxantrone'], axis = 1, inplace = True)
# train_pd_df.drop(['mycophenolate'], axis = 1, inplace = True)
# train_pd_df.drop(['interferons'], axis = 1, inplace = True)
# train_pd_df.drop(['alkylating agent'], axis = 1, inplace = True)
# train_pd_df.drop(['hydroxyurea'], axis = 1, inplace = True)

# train_pd_df.drop(['dapsone'], axis = 1, inplace = True)
train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-6 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['IL-23 inhibitor'], axis = 1, inplace = True)
train_pd_df.drop(['anti-BLyS'], axis = 1, inplace = True)
# train_pd_df.drop(['S1P receptor modulator'], axis = 1, inplace = True)
train_pd_df.drop(['integrin inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['PDE4i targeted synthetic'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD20'], axis = 1, inplace = True)
train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)

# train_pd_df.drop(['mercaptopurine'], axis = 1, inplace = True)
# train_pd_df.drop(['abatacept'], axis = 1, inplace = True)

# train_pd_df.drop(['IL-1 inhibitor'], axis = 1, inplace = True)
# train_pd_df.drop(['cladribine'], axis = 1, inplace = True)
# train_pd_df.drop(['anti-CD52'], axis = 1, inplace = True)
# train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)


percent_missing = train_pd_df.isnull().sum() * 100 / len(train_pd_df)
missing_value_df = pd.DataFrame({'column_name': train_pd_df.columns,
                                 'percent_missing': percent_missing})

display(missing_value_df)

## Notice! Need to comment out once have the initial VIF results
## Fill NA values
## Method 1: Remove rows with at least one null value
# train_pd_noNull_df = train_pd_df.dropna()
# train_pd_df = train_pd_noNull_df

## Notice
## Remove mAbs, antibody feature here
train_pd_df.drop(['monoclonal antibody covid-19'], axis = 1, inplace = True)

column_name,percent_missing
hospitalized_after_positive,0.0
IMV_after_positive,0.0
death_after_positive,0.0
age,0.0
BMI,0.0
sex:male,0.0
hypertension,0.0
diabetes (type 1+2),0.0
atrial fibrillation,0.0
coronary artery disease,0.0


In [0]:
## manually set the random seed to define a replication
r_seed = 42
print("current random seed is: ", r_seed)

## manually set the number for cross validation
num_cv = 10
print("current CV fold selection is: ", num_cv)

## Possible response vectors
Y_cols = ["hospitalized_after_positive", 'IMV_after_positive', 'death_after_positive']

## separate X and Y
train_df_Y = train_pd_df[Y_cols]
train_df_X = train_pd_df.drop(Y_cols, axis=1)

## Create composite response vectors
train_df_Y_new = pd.DataFrame(train_df_Y, columns = Y_cols)

train_df_Y_new['hospitalized_or_IMV_or_death'] = train_df_Y['hospitalized_after_positive'] + train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']
train_df_Y_new['IMV_or_death'] = train_df_Y['IMV_after_positive'] + train_df_Y['death_after_positive']

## Convert those >1 values back to 1
train_df_Y_new.loc[train_df_Y_new['hospitalized_or_IMV_or_death'] >= 1, 'hospitalized_or_IMV_or_death'] = 1
train_df_Y_new.loc[train_df_Y_new['IMV_or_death'] >= 1, 'IMV_or_death'] = 1

current random seed is:  42
current CV fold selection is:  10


In [0]:
## Notice: LR need a minimum of 10 events (>50 better) per independent variable
## Therefore, the following features need to be excluded for lack of events

# train_df_X.iloc[:,2:].apply(pd.Series.value_counts)

In [0]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# # VIF dataframe
# vif_data = pd.DataFrame()
# vif_data["feature"] = train_df_X.columns

# # calculating VIF for each feature
# vif_data["VIF"] = [variance_inflation_factor(train_df_X.values, i) for i in range(len(train_df_X.columns))]

# display(vif_data)

In [0]:
# import matplotlib.pyplot as plt
# from bioinfokit import visuz

# visuz.stat.corr_mat(df=train_df_X, cmap='RdBu',dim = (15,15),  show=True)

In [0]:
## Found workaround at blog: this https://stackoverflow.com/questions/71106940/cannot-import-name-centered-from-scipy-signal-signaltools
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [0]:
# training a linear SVM classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
import lxml

####################################################################################################
## Method used to do univariate logistic regression on each single feature/variable
## Inputs:
## 
## Output:
## 
######################################################################################################
def multivariate_KNN_model(X_train, y_train, X_test, y_test):
  ## Instantiate a bionomial family and logit link function GLM model as logistic regression model
  model = KNeighborsClassifier(n_neighbors = 10, weights = 'uniform', algorithm = 'auto').fit(X_train, y_train)
  model_binaryclass_auroc = binary_class_roc_auc_score(y_test, model.predict(X_test))

  print("Auroc on test data set: %0.5f" % (model_binaryclass_auroc))
  
  return

## Response == "death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'death_after_positive'
Y = train_df_Y[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['death_after_positive'])

unique labels from y:  [0 1]


In [0]:
multivariate_KNN_model(os_data_X, os_data_y, X_test, y_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Auroc on test data set: 0.71533


## Response == "hospitalized_or_IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'hospitalized_or_IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['hospitalized_or_IMV_or_death'])

unique labels from y:  [0 1]


In [0]:
multivariate_KNN_model(os_data_X, os_data_y, X_test, y_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Auroc on test data set: 0.63832


## Response == "IMV_or_death"

In [0]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

###############################################################################
## Current Y as response
## Notice: need manual check
## Possible options: hospitalized, invasive_mechanical_vent, death, results
################################################################################

select_col = 'IMV_or_death'
Y = train_df_Y_new[select_col]
# Y = Y.map(dict(yes=1, no=0))

class_names = np.unique(Y)
print("unique labels from y: ", class_names)

## Train test split use r_seed assigned in CMD 1
X = train_df_X
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=r_seed, stratify = Y)
columns = X_train.columns

## Normalize after the train/test split to avoid any potential data leakage
## using the min and max from the trainset to do minmax on test set
age_min, age_max = X_train["age"].min(), X_train["age"].max()
X_train["age"] = (X_train["age"] - age_min) / (age_max - age_min)
X_test["age"] = (X_test["age"] - age_min) / (age_max - age_min)

BMI_min, BMI_max = X_train["BMI"].min(), X_train["BMI"].max()
X_train["BMI"] = (X_train["BMI"] - BMI_min) / (BMI_max - BMI_min)
X_test["BMI"] = (X_test["BMI"] - BMI_min) / (BMI_max - BMI_min)

## Random oversampling using random seed
## Define oversampling strategy: default, 'auto': equivalent to 'not majority'
oversample = RandomOverSampler(random_state=r_seed)
os_data_X, os_data_y = oversample.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X, columns=columns)
os_data_y= pd.DataFrame(data=os_data_y, columns=['IMV_or_death'])

unique labels from y:  [0 1]


In [0]:
multivariate_KNN_model(os_data_X, os_data_y, X_test, y_test)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Auroc on test data set: 0.70139
